# processMeerKAT Jupyter Notebook Tutorial

processMeerKAT implements a CASA based wide-band full Stokes calibration pipeline (in the linear basis). Broadly, the pipeline aims to “do the right thing” and by keeping the steps as general as possible we believe that there should be no need for fine tuning in order to obtain a well calibrated dataset.

This Jupyter Notebook Tutorial is based on the [processMeerKAT](https://idia-pipelines.github.io/docs/processMeerKAT) documentation for the IDIA processMeerKAT Pipeline.

In [ ]:
casa['build']['version']

In [ ]:
from __future__ import print_function
import sys
import os
import subprocess

home = !pwd

## singularity data path:
#data_path = os.path.join(home[0],'data/')

## singularity pipeline path:
pipeline_path = os.path.join(home[0],'pipelines/processMeerKAT/')
functions_path = os.path.join(home[0],'pipelines/processMeerKAT/cal_scripts/')

## singularity module paths:
sys.path.append(pipeline_path)
sys.path.append(functions_path)


In [ ]:
### View config file

f = open('myconfig.txt', 'r')
file_contents = f.read()
print(file_contents)
f.close()

## Input validation 

This script performs a few basic validity checks, on the default config file, and on the input MS. the existence of the input MS, and the data types of the inputs specified in the config file are all verified before the pipeline continues to the next steps. If reference antenna calculation is not requested, a simple check is performed to verify that the input reference antenna exists in the MS. Otherwise, the following paragraph describes the details of reference antenna calculation.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/validate_input.py --config myconfig.txt

# Reference antenna calculation 

If the `calcrefant` parameter in the config file is set to `True`, then this script is executed. The algorithm works by calculating the median and standard deviation over all the visibility amplitudes for a given antenna, and iterates over every antenna in the array. Any outlier antennas, in the top 2 and bottom 5 percentile of this distribution are then flagged. The reference antenna is selected to be the un-flagged antenna with the smallest visibility rms.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/calc_refant.py --config myconfig.txt

# Data partition

The input measurement set (MS) is partitioned into a [multi-measurement set (MMS)](https://casa.nrao.edu/casadocs/casa-5.4.1/uv-manipulation/data-partition) using the CASA task `partition`. This task splits up the main MS into smaller SUBMSs that are individual units of a larger logical MMS. The number of SUBMSs created are equal to the number of scans in the input MS. Partitioning the data in this manner allows for more efficient use of computation while using MPI, since each SUBMS can be independently operated on by different MPI workers.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/partition.py --config myconfig.txt

# Flagging (round 1) 

The first of two rounds of pre-calibration flagging. If `badfreqranges` and `badants` are specified in the config file, they are flagged. These lists are also allowed to be empty. Further, any autocorrelations are also flagged using `mode='manual'` and `autocorr=True` in the flagdata parameters. 

Subsequently, `flagdata` is called on the calibrators and target sources with conservative limits to clip out the worst RFI (the data are clipped at the level of 50 Jy). It also makes a single call to `tfcrop` to flag data at a 6 $\sigma$ limit. `tfcrop` in this case is preferred, since the as yet uncalibrated bandpass shape should be taken care of by fitting a piecewise polynomial across the band.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/flag_round_1.py --config myconfig.txt

# setjy 

The `setjy` task is run on the specified primary calibrators - this step is run once each before the first and second rounds of calibration.

By default, the ‘Perley-Butler 2010’ flux scale is used, since it is the only one which contains the popular southern calibrator PKS B1934-638. In case the calibrator J0408-6545 is present in the data, it is preferred. A broadband Stokes I model for J0408-6545 is used, via the `manual` mode of `setjy`.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/setjy.py --config myconfig.txt

# Parallel hand calibration 

Standard delay, bandpass and gain calibration is run on the data, in order to obtain better statistics for a second round of flagging.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/xx_yy_solve.py --config myconfig.txt

In [ ]:
run pipelines/processMeerKAT/cal_scripts/xx_yy_apply.py --config myconfig.txt

# Flagging (round 2) 

Similar to the first round, the `tfcrop` algorithm is run independently on the primary and secondary calibrator and the target(s). The thresholds are lower than the first round as the algorithm is now operating on calibrated data.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/flag_round_2.py --config myconfig.txt

In [ ]:
run pipelines/processMeerKAT/cal_scripts/setjy.py --config myconfig.txt

# Cross hand calibration 

The full Stokes calibration procedure is done across as much of the SPW as is requested in the config file. In the default case, the entire SPW (spanning ~ 800 MHz) is calibrated across. The caveat here is that CASA does not support a true wideband, full polarization calibration. For example the Stokes Q and U values of a source with non-zero RM across the band will not be correctly accounted for. The assumption CASA makes is that the bandwidth is split into several smaller SPWs (such as is the case of VLA or ALMA) and that the Stokes parameters within each SPW can be assumed to be a constant. We have identified work-arounds to this, and will be implementing the fix in upcoming versions of the pipeline.

The cross-hand calibration performs the following steps:
* Delay calibration (the K term), time averaged, parallel hand
* Bandpass calibration (the B term), time averaged, parallel hand
* Cross hand delay calibration (the KCROSS term), time averaged, cross hand

After the cross-hand delay calibration is performed, we iterate over calculating the time dependent gains. Initially the time-dependent gains are calculated for the primary and secondary calibrators, as a function of time and parallactic angle. The polarization properties of the secondary are assumed to be unknown, and are determined from the gain variation as a function of parallactic angle. This is fit for by the `qufromgain` task, which is contained in `almapolhelpers` and can be accessed in CASA by

```
from almapolhelpers import *
```

This imports several helper tasks that are meant to solve ALMA polarization, but are general enough to work with any telescope that has linear feeds.

Once the fractional Q and U values are determined for the phase calibrator, the gain solutions are recomputed with the fractional polarization as an input, in theory resulting in more accurate gain solutions. This is followed by a call to `xyamb`, also within `almapolhelpers` that breaks the ambiguity in the X-Y phases for the solutions generated by `qufromgain`. These can be cross-checked with the solutions obtained by running `gaincal` with `gaintype='XYf+QU'` which solves for the X-Y phase as a function of frequency, assuming an unknown source Q, U value. Finally we run `polcal` in the `Dflls` mode in order to calculate the polarization leakage (the D term) as a function of frequency (f), using a linear least squares algorithm (lls). Finally, we bootstrap the fluxes from the primary to the secondary using `fluxscale`.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/xy_yx_solve.py --config myconfig.txt

In [ ]:
run pipelines/processMeerKAT/cal_scripts/xy_yx_apply.py --config myconfig.txt

# Splitting out calibrated data 

Finally the calibrated data are averaged down in time and frequency by the amount specified in the config file, and the target(s) and calibrators are split out into separate MMSs for further imaging/processing.

In [ ]:
run pipelines/processMeerKAT/cal_scripts/split.py --config myconfig.txt

In [ ]:
run pipelines/processMeerKAT/cal_scripts/quick_tclean.py --config myconfig.txt

In [ ]:
run pipelines/processMeerKAT/cal_scripts/plot_solutions.py --config myconfig.txt